Assumption: This is not a time series problem. The dates are irrelevant. To be explored more

In [44]:
from RedStockData import merged
import numpy as np
import pandas as pd

The best target variable would be "open" - "close", which reflects the influence that reddit exerts in a given day.

In [114]:
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

day_open = merged[merged.columns[-6]]
day_close = merged[merged.columns[-3]]
for i in range(len(merged)):
    merged["Day_Change"][i] = format((day_open[i]/day_close[i])-1, '.5f')

#### Get X and y

In [115]:
merged.head(3)

,date,total_posts,score,awards,authors,threads,clov,sofi,wkhs,amd,gme,x,amc,clne,nio,mu,spce,bb,symbol,open,high,low,close,volume,Day_Change
0,2014-04-10,19,15,0.0,10,5,0,0,0,0,0,0,0,0,0,0,0,0,gme,23.59,23.640,22.88,22.88,118000,0.03103
1,2014-04-11,65,173,0.0,35,11,0,0,0,0,0,0,0,0,0,0,0,0,gme,22.73,23.710,21.91,23.60,581500,-0.03686
2,2014-07-30,20,41,0.0,13,7,0,0,0,0,0,1,0,1,0,0,0,0,gme,22.65,22.759,22.30,22.51,153600,0.00622


In [116]:
X = merged.iloc[:, 1:18] # Excluding Date, from "total_posts" to before "symbol"

y = merged["Day_Change"]

PCA Model - This is my first step of using Machine learning. There are many ways to go about this, but I'm just trying this way as a first attempt.

In [118]:
from sklearn.decomposition import PCA

pca = PCA(n_components=4)
X_pca = pca.fit_transform(X)
PCA_df = pd.DataFrame(data = X_pca)
PCA_df["y"] = y
PCA_df.head()

,0,1,2,3,y
0,-3708.332098,-656.835331,4.534147,28.547481,0.03103
1,-3545.097969,-624.609211,-2.054066,26.412853,-0.03686
2,-3682.177695,-657.478926,6.362985,27.174459,0.00622
3,-3673.561021,-653.076974,5.846454,25.698310,0.02694
4,-3707.852812,-661.018097,4.146595,28.028418,-0.01391


In [155]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [219]:
# Automated selection of the number of features, (as well as train-test splits, since that is rather a data factor, not a modeling factor) in order to get the best R^2 score
scores = []
for PCs in range(1,15,1):
    for split in range(3,7,1):
        pca = PCA(n_components=PCs)
        X_pca = pca.fit_transform(X)
        PCA_df = pd.DataFrame(data = X_pca)
        X_train, X_test = X_pca[0:int(len(X)*split/10)], X_pca[int(len(X)*split/10):]
        y_train, y_test = y[0:int(len(y)*split/10)], y[int(len(y)*split/10):]
        model = model.fit(X_train, y_train)
        scores.append(
            [PCs, split/10, model.score(X_test,y_test)]
        )

scores = pd.DataFrame(scores)
scores[scores[scores.columns[2]]==scores[scores.columns[2]].max()]

,0,1,2
2,1,0.5,0.015732


#### With 1 principal component, and half way split, the model returns a 0.157 R^2 value with the given data

#### Now we try another model selection methodd

In [232]:
from sklearn.svm import SVC
from sklearn.feature_selection import RFE

RFE(estimator=SVC(C=1, kernel='linear'), n_features_to_select=1)

In [236]:
# Automated selection (as described above)
scores = []
for PCs in range(1,15,1):
    for split in range(30,70,1):
        svc = SVC(kernel="linear", C=1)
        X_pca = pca.fit_transform(X)
        rfe = RFE(estimator=svc, n_features_to_select=1, step=1)
        X_train, X_test = X[0:int(len(X)*split/100)], X[int(len(X)*split/100):]
        y_train, y_test = y[0:int(len(y)*split/100)], y[int(len(y)*split/100):]
        model = rfe.fit(X_train, y_train)
        scores.append(
            [PCs, split/10, model.score(X_test,y_test)]
        )

scores = pd.DataFrame(scores)
scores[scores[scores.columns[2]]==scores[scores.columns[2]].max()]

,0,1,2
21,1,5.1,0.013514
61,2,5.1,0.013514
101,3,5.1,0.013514
141,4,5.1,0.013514
181,5,5.1,0.013514
221,6,5.1,0.013514
261,7,5.1,0.013514
301,8,5.1,0.013514
341,9,5.1,0.013514
381,10,5.1,0.013514


The best R2 results from this model is 0.0135